In [79]:
%pip install unidecode


[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [80]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import unidecode
import seaborn as sns

In [81]:
df = pd.read_csv('../data/raw-props-pinamar.csv')
#df = pd.read_csv('../data/raw-pinamar-test.csv')

df = df.rename(columns={
    'posting-prices-module--price': 'raw_precio',
    'posting-locations-module--location-address': 'direccion',
    'posting-card-module--posting-description': 'descripcion',
    'pills-module--trigger-pill-item-span': 'caracteristica', 
    'posting-main-features-module--posting-main-features-span': 'feat_1',
    'posting-main-features-module--posting-main-features-span_2': 'feat_2',
    'posting-main-features-module--posting-main-features-span_3': 'feat_3',
    'posting-main-features-module--posting-main-features-span_4': 'feat_4',
    'posting-main-features-module--posting-main-features-span_5': 'feat_5'
})

# Juntar feat_1 a feat_5 para buscar caracteristicas
df['features_text'] = (
    df['feat_1'].fillna('') + " " + 
    df['feat_2'].fillna('') + " " + 
    df['feat_3'].fillna('') + " " + 
    df['feat_4'].fillna('') + " " + 
    df['feat_5'].fillna('')
)


# LIMPIEZA  PRECIO
df['raw_precio'] = df['raw_precio'].astype(str) 
df['precio'] = df['raw_precio'].str.replace('USD', '', regex=False).str.replace('.', '', regex=False).str.strip()
df['precio'] = pd.to_numeric(df['precio'], errors='coerce')

def extraer_numero(texto, patron):
    match = re.search(patron, str(texto).lower())
    if match:
        return float(match.group(1))
    return 0.0 

# extracción sobre features_text
df['m2'] = df['features_text'].apply(lambda x: extraer_numero(x, r'(\d+)\s*m²'))
df['ambientes'] = df['features_text'].apply(lambda x: extraer_numero(x, r'(\d+)\s*amb'))
df['dormitorios'] = df['features_text'].apply(lambda x: extraer_numero(x, r'(\d+)\s*dorm'))
df['baños'] = df['features_text'].apply(lambda x: extraer_numero(x, r'(\d+)\s*bañ'))
df['cocheras'] = df['features_text'].apply(lambda x: extraer_numero(x, r'(\d+)\s*coch'))

df['caracteristica'] = df['caracteristica'].fillna('Sin etiqueta')

cols_finales = [
    'precio', 'm2', 'ambientes', 'dormitorios', 'baños', 'cocheras', 
    'direccion', 'descripcion', 'caracteristica', 'features_text'
]

df_final = df[cols_finales].dropna(subset=['precio', 'm2']).copy()

df_final = df_final[df_final['m2'] > 0]

print(f"{len(df_final)} propiedades")
df_final.head()

1429 propiedades


,precio,m2,ambientes,dormitorios,baños,cocheras,direccion,descripcion,caracteristica,features_text
0,1350000.0,1125.0,10.0,8.0,7.0,3.0,Barrio Maritimo 1,Chalet Minimalista de Categoria 10 amb c/ vist...,Pileta,1125 m² tot. 10 amb. 8 dorm. 7 baños 3 coch.
1,1950000.0,1458.0,5.0,4.0,3.0,6.0,De la Cincha 400,Casa ubicada sobre el lago en la calle De la C...,Pileta,1458 m² tot. 5 amb. 4 dorm. 3 baños 6 coch.
2,510000.0,2260.0,6.0,4.0,3.0,2.0,de las Driza al 1400,"Excelente casa, emplazada sobre un lote de más...",Quincho,2260 m² tot. 6 amb. 4 dorm. 3 baños 2 coch.
4,570000.0,1239.0,6.0,5.0,4.0,4.0,Artemisa al 300,Epic Real Estate presenta esta casa a la venta...,Terraza,1239 m² tot. 6 amb. 5 dorm. 4 baños 4 coch.
5,110000.0,300.0,4.0,2.0,2.0,0.0,Constitucion al 2100,Lindisimo ph en Pinamar. Al ingresar al terren...,Dormitorio en suite,300 m² tot. 4 amb. 2 dorm. 2 baños


In [82]:
print(f"Filas antes: {len(df_final)}")

df_final = df_final.drop_duplicates(subset=['descripcion', 'precio'], keep='first')

df_final = df_final.reset_index(drop=True)

print(f"Filas despues: {len(df_final)}")

duplicados_restantes = df_final.duplicated(subset=['descripcion', 'precio']).sum()
print(f"Duplicados restantes: {duplicados_restantes} ")

Filas antes: 1429
Filas despues: 1288
Duplicados restantes: 0 


In [83]:
numeros_texto = {
    'un': 1, 'uno': 1, 'una': 1, 'dos': 2, 'tres': 3, 
    'cuatro': 4, 'cinco': 5, 'seis': 6, 'doble': 2, 'triple': 3,  'primer': 1, 'doble': 2, 'segundo': 2,
     'triple': 3, 'tercer': 3, 'cuadruple': 4
}

def rescate_inteligente(texto, regex_raiz, regex_existencia=None):
    if pd.isna(texto): return 0.0
    texto = str(texto).lower()
    
    # Busca numeros explicitos 
    match_digit = re.search(r'(\d+)\s*[\.\-]?\s*' + regex_raiz, texto)
    if match_digit:
        return float(match_digit.group(1))
        
    # Busca numeros en letras 
    for palabra, valor in numeros_texto.items():
        if re.search(f"{palabra}\\s+{regex_raiz}", texto):
            return float(valor)

    # Buscamos existencia (ej: dice garage pero no cuantos)
    if regex_existencia:
        if re.search(r'(sin|no tiene|no posee)\s*' + regex_existencia, texto):
            return 0.0
        
        if re.search(regex_existencia, texto):
            return 1.0 
            
    return 0.0

# unir descripción y features para buscar en ambos 
df_final['texto_completo'] = df_final['descripcion'].fillna('') + " " + df_final['features_text'].fillna('')

# Solo aplica el rescate si el valor actual es 0
mask_dorm = df_final['dormitorios'] == 0
df_final.loc[mask_dorm, 'dormitorios'] = df_final.loc[mask_dorm, 'texto_completo'].apply(
    lambda x: rescate_inteligente(x, r'(dorm|habitaci)', r'(dorm|habitaci|cámara)')
)

mask_banos = df_final['baños'] == 0
df_final.loc[mask_banos, 'baños'] = df_final.loc[mask_banos, 'texto_completo'].apply(
    lambda x: rescate_inteligente(x, r'bañ', r'(bañ|toil|sanitario)')
)

mask_coch = df_final['cocheras'] == 0
df_final.loc[mask_coch, 'cocheras'] = df_final.loc[mask_coch, 'texto_completo'].apply(
    lambda x: rescate_inteligente(x, r'(coch|gara)', r'(cochera|garage|garaje|coch\.|gge\.)')
)

print(f"Total filas: {len(df_final)}")
print("Ceros restantes después del rescate:")
print((df_final[['dormitorios', 'baños', 'cocheras', 'm2', 'dormitorios', 'ambientes']] == 0).sum())


Total filas: 1288
Ceros restantes después del rescate:
dormitorios      2
baños            5
cocheras       241
m2               0
dormitorios      2
ambientes      120
dtype: int64


In [84]:
def rescate_m2_estricto(texto):
    if pd.isna(texto): return 0.0
    texto = str(texto).lower().replace('.', '') 
    
    # Busca patrones como: "200 m2", "200 mts", "200 metros", "superficie 200"
    match = re.search(r'(\d+)\s*(m2|mts|metros|cubiertos|sup\.|totales)', texto)
    
    if match:
        valor = float(match.group(1))
        # Filtro de cordura: Si extrae un año o un precio, lo ignora
        if 10 < valor < 10000:
            return valor
            
    return 0.0

numeros_texto = {'un': 1, 'uno': 1, 'una': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, 'siete': 7, 'ocho': 8}

def rescate_ambientes_v2(row):
    texto = str(row['texto_completo']).lower()
    
    if 'monoamb' in texto: return 1.0
        
    match_digit = re.search(r'(\d+)\s*(ambientes|amb\b)', texto)
    if match_digit: return float(match_digit.group(1))
    
    for palabra, valor in numeros_texto.items():
        if re.search(f"{palabra}\\s+(ambientes|amb\\b)", texto):
            return float(valor)

# Calcula la mediana de ambientes para cada cantidad de dormitorios
referencia_dorms = df_final[df_final['ambientes'] > 0].groupby('dormitorios')['ambientes'].median()

# Convertir a diccionario 
mapa_ambientes = referencia_dorms.to_dict()

# IMPUTACIÓN INTELIGENTE
def completar_ambientes_smart(row):
    # Si ya tiene valor (y no es 0), devolver intacto
    if row['ambientes'] > 0:
        return row['ambientes']
    
    dorms = row['dormitorios']
    m2 = row['m2']
    precio = row['precio']
    
    # BASADO EN DORMITORIOS
    if dorms > 0:
        # Si tiene 3 dorms, pone lo que la mayoría de casas de 3 dorms tiene
        if dorms in mapa_ambientes:
            return mapa_ambientes[dorms]
        else:
            # Si tiene 10 dorms y no está en el mapa, usa la regla simple
            return dorms + 1.0

    # BASADO EN METROS CUADRADOS
    if m2 > 0:
        if m2 < 45: return 1.0   # Monoambiente
        if m2 < 70: return 2.0   # 1 Dormitorio + Living
        if m2 < 100: return 3.0  # 2 Dormitorios + Living
        if m2 >= 100: return 4.0 # Casa grande
        
    #BASADO EN PRECIO
    if precio > 0:
        if precio < 60000: return 2.0
        if precio < 120000: return 3.0
        return 4.0

    return 0.0 # Si falló todo
mask_amb_cero = df_final['ambientes'] == 0
print(f"Aplicando inteligencia a {mask_amb_cero.sum()}")
df_final.loc[mask_amb_cero, 'ambientes'] = df_final[mask_amb_cero].apply(completar_ambientes_smart, axis=1)

# APLICACIÓN
mask_amb = df_final['ambientes'] == 0
print(f"Intentando rescatar {mask_amb.sum()} ceros")

df_final.loc[mask_amb, 'ambientes'] = df_final[mask_amb].apply(rescate_ambientes_v2, axis=1)

print(f"Ceros restantes en Ambientes: {(df_final['ambientes'] == 0).sum()}")

df_final['texto_completo'] = df_final['descripcion'].fillna('') + " " + df_final['features_text'].fillna('')

mask_m2 = df_final['m2'] == 0
print(f"Intentando rescatar m2 en {mask_m2.sum()} propiedades...")
df_final.loc[mask_m2, 'm2'] = df_final.loc[mask_m2, 'texto_completo'].apply(rescate_m2_estricto)

# RESULTADOS
print("ESTADO FINAL")
print(f"M2 faltantes: {(df_final['m2'] == 0).sum()}")
print(f"Ambientes faltantes: {(df_final['ambientes'] == 0).sum()}")

Aplicando inteligencia a 120
Intentando rescatar 0 ceros
Ceros restantes en Ambientes: 0
Intentando rescatar m2 en 0 propiedades...
ESTADO FINAL
M2 faltantes: 0
Ambientes faltantes: 0


/var/folders/_3/9ky5qgzn2f733_p25r49fl700000gn/T/ipykernel_75487/4199483276.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[mask_m2, 'm2'] = df_final.loc[mask_m2, 'texto_completo'].apply(rescate_m2_estricto)


In [85]:
def detectar_estado_malo(texto):
    if pd.isna(texto): return 0 
    texto = str(texto).lower()
    
    regex_malo = r'(a refaccionar|para refaccionar|a reciclar|para reciclar|a reparar|falta mantenimiento|humedad|demoler|demolición|valor lote|valor terreno|solo terreno|inhabitable|precari)'
    
    #Anti-Falsos Positivos / Si dice "reciclado a nuevo", NO es estado malo
    regex_bueno_explicito = r'(totalmente reciclado|reciclado a nuevo|impecable|a estrenar|refaccionado a nuevo)'

    if re.search(regex_malo, texto) and not re.search(regex_bueno_explicito, texto):
        return 1
        
    return 0

df_final['texto_completo'] = df_final['descripcion'].fillna('') + " " + df_final.get('features_text', pd.Series([""] * len(df_final))).fillna('')
df_final['es_a_refaccionar'] = df_final['texto_completo'].apply(detectar_estado_malo)


# VERIFICACION
conteo = df_final['es_a_refaccionar'].value_counts()
print(f"Props buen estado (0): {conteo.get(0, 0)}")
print(f"Props a refaccionar/demoler (1): {conteo.get(1, 0)}")

Props buen estado (0): 1276
Props a refaccionar/demoler (1): 12


In [86]:
def detectar_amenity_minucioso(texto, amenity_regex):
    if pd.isna(texto): return 0.0
    texto = str(texto).lower()
    
    # si la palabra no está en todo el texto, devolvemos 0
    if not re.search(amenity_regex, texto):
        return 0.0

    # Dividir el texto en bloques usando puntuación 
    bloques = re.split(r'[.;\n•\-!]', texto)

    # Palabras que, si aparecen antes del amenity en el MISMO bloque, anulan su existencia
    negadores = [
        r'ideal\s', r'espacio\s', r'lugar\s', r'posibilidad\s',
        r'proyecto\s', r'futur[oa]', r'apto\s', r'permite\s',
        r'preparad[oa]\s', r'opci[oó]n\s', r'sin\s', r'no posee\s',
        r'fondo\s+para', r'ampliaci[oó]n', r'construcci[oó]n', 
        r'a\s+terminar' # Ej: "Quincho a terminar" = No tiene quincho funcional
    ]

    regex_negadores = r'(' + '|'.join(negadores) + r')'

    for bloque in bloques:
        bloque = bloque.strip()
        
        if not re.search(amenity_regex, bloque):
            continue
            
        # .{0,60}: Busca el negador y permite hasta 60 caracteres de distancia hasta el amenity.
        patron_trampa = f"{regex_negadores}.{{0,60}}{amenity_regex}"
        
        if re.search(patron_trampa, bloque):
            continue 
        
        return 1.0
    return 0.0


df_final['texto_completo'] = df_final['descripcion'].fillna('') + " . " + df_final.get('features_text', pd.Series([""] * len(df_final))).fillna('')

df_final['tiene_pileta'] = df_final['texto_completo'].apply(
    lambda x: detectar_amenity_minucioso(x, r'(pileta|piscina|natatorio|jacuzzi)')
)

df_final['tiene_quincho'] = df_final['texto_completo'].apply(
    lambda x: detectar_amenity_minucioso(x, r'(quincho|parrillero techado|churrasquera)')
)

print(f" Piletas: {df_final['tiene_pileta'].sum()}")
print(f" Quinchos: {df_final['tiene_quincho'].sum()}")

 Piletas: 500.0
 Quinchos: 291.0


In [87]:
casos_cero = df_final[
    (df_final['dormitorios'] == 0) | 
    (df_final['baños'] == 0)
].copy()

print(f" {len(casos_cero)} propiedades con 0 habitaciones o baños")
cols_ver = ['precio', 'm2', 'dormitorios', 'baños', 'descripcion']

for i, row in casos_cero.sort_values('precio').iterrows():
    print(f"ID {i} | USD {row['precio']:,.0f} | {row['m2']:.0f} m² ")
    print(f"Dormitorios: {row['dormitorios']} | Baños: {row['baños']}")
    print(row['descripcion'])

 6 propiedades con 0 habitaciones o baños
ID 1269 | USD 150,000 | 100 m² 
Dormitorios: 2.0 | Baños: 0.0
Venta de Casa 2 dormitorios, pinamar! xintel(dit-dt1-5932)
ID 472 | USD 380,000 | 216 m² 
Dormitorios: 2.0 | Baños: 0.0
Venta de Casa, Pinamarxintel(bml-bml-1155)
ID 852 | USD 395,000 | 250 m² 
Dormitorios: 0.0 | Baños: 2.0
Venta de Casa 4 ambientes en La Herradura, Pinamar. El barrio La Herradura Pinamar es un barrio consolidado que posee amplios lotes de un promedio de 1400 m2 que dan amplitud y privacidad a sus propietarios. El barrio cuenta con cuatro lagos, dos canchas de polo, club Hípico, Canchas de Tenis y Futbol, seguridad las 24 hs con rondines y guardia en sus dos entradas permanente. La Casa se encuentra desarrollada en una planta, cuenta con tres habtaciones una en suite y dos compraten baño, living comedor cocina integrados, toillete y galeria cerrada con parrilla tipo quincho y garage para cuatri. Las medidas expresadas en esta publicación son al solo efecto orientativ

In [88]:
# Pasar los 0 a NaN temporalmente para cuidar los promedios matemáticos
df_final['dormitorios'] = df_final['dormitorios'].replace(0, np.nan)
df_final['baños'] = df_final['baños'].replace(0, np.nan)

# IMPUTAR DORMITORIOS (Basado en los ambientes)/ Calcula la mediana de dormitorios para cada grupo de ambientes
dorm_por_ambiente = df_final.groupby('ambientes')['dormitorios'].transform('median')
mediana_dorm_global = df_final['dormitorios'].median()
df_final['dormitorios'] = df_final['dormitorios'].fillna(dorm_por_ambiente).fillna(mediana_dorm_global)

# IMPUTAR BAÑOS (Basado en los dormitorios que acabamos de calcular)/ Calcula la mediana de baños para cada grupo de dormitorios
banos_por_dorm = df_final.groupby('dormitorios')['baños'].transform('median')
mediana_banos_global = df_final['baños'].median()
df_final['baños'] = df_final['baños'].fillna(banos_por_dorm).fillna(mediana_banos_global)

# Limpieza final: Redondea y pasa a número entero
df_final['dormitorios'] = df_final['dormitorios'].round().astype(int)
df_final['baños'] = df_final['baños'].round().astype(int)

print("Ceros restantes:")
print((df_final[['dormitorios', 'baños', 'cocheras']] == 0).sum())

Ceros restantes:
dormitorios      0
baños            0
cocheras       241
dtype: int64


In [89]:
def extraer_cubiertos(texto):
    if pd.isna(texto): return None
    t = str(texto).lower()
    
    # Normalizamos el texto
    t = t.replace('mts', 'm').replace('mts2', 'm2').replace('metros', 'm')

    # Patrones de búsqueda
    patrones = [
        r'(\d+(?:[\.,]\d+)?)\s*m2?\s*(?:cubiertos|cub|c\.|construidos|edificados)',
        r'(?:sup|superficie)\.?\s*(?:cubierta|cub|edificada|construida).*?(\d+(?:[\.,]\d+)?)',
        r'(?:construidos|edificados).*?(\d+(?:[\.,]\d+)?)'
    ]
    
    for pat in patrones:
        match = re.search(pat, t)
        if match:
            valor_str = match.group(1).replace(',', '.')
            try:
                valor = float(valor_str)
                if 20 < valor < 1000: return valor
            except:
                continue
    return None

columna_texto = 'texto_analisis' if 'texto_analisis' in df_final.columns else 'descripcion'
df_final['m2_cubiertos_extract'] = df_final[columna_texto].apply(extraer_cubiertos)

# FUNCIÓN DE CONSOLIDACIÓN
def consolidar_m2(row):
    cubiertos = row['m2_cubiertos_extract']
    totales = row.get('m2', 0)
    
    # Si encuentra el dato exacto en el texto -> Usa ese
    if pd.notna(cubiertos):
        if cubiertos > totales * 1.5 and totales > 10: 
            return totales # Corrección de error si cubiertos > terreno
        return cubiertos
    
    # Si ess departamento -> Asume Total = Cubierto
    if row.get('es_ph_depto', 0) == 1:
        return totales 
        
    # Si es CASA y no dice cubiertos -> ESTIMACIÓN POR COMPONENTES
    M2_BASE_SOCIAL = 35    # Living/Cocina/Comedor
    M2_DORMITORIO = 14     # Dormitorio + Placar + Circulación
    M2_BANO = 6            # Baño
    M2_COCHERA = 18        # Cochera
    M2_QUINCHO = 30        # Quincho
    
    ambientes = row.get('ambientes', 0)
    if pd.isna(ambientes): ambientes = 3 # Promedio por defecto
         
    # Si no tiene dato de dormitorios, estima: Ambientes - 1 (Living)
    dormitorios = row.get('dormitorios', 0)
    if pd.isna(dormitorios) or dormitorios == 0:
        dormitorios = max(1, ambientes - 1)
        
    banos = row.get('baños', 1)
    if pd.isna(banos): banos = 1
    
    cocheras = row.get('cocheras', 0)
    if pd.isna(cocheras): cocheras = 0
    
    tiene_quincho = row.get('tiene_quincho', 0)
    if pd.isna(tiene_quincho): tiene_quincho = 0
    
    estimacion = M2_BASE_SOCIAL + \
                 (dormitorios * M2_DORMITORIO) + \
                 (banos * M2_BANO) + \
                 (cocheras * M2_COCHERA)
                 
    if tiene_quincho == 1:
        estimacion += M2_QUINCHO
        
    # Validación de seguridad: La casa no puede ser más grande que el terreno (deja margen para paredes/patio mín)
    if estimacion > totales and totales > 10:
        return totales * 0.90 
        
    return estimacion
df_final['m2_construidos_final'] = df_final.apply(consolidar_m2, axis=1)

print(f"Datos extraídos del texto: {df_final['m2_cubiertos_extract'].notna().sum()}")
print(f"Datos calculados por ambientes: {df_final['m2_cubiertos_extract'].isna().sum()}")

Datos extraídos del texto: 268
Datos calculados por ambientes: 1020


In [90]:
df_final['texto_analisis'] = df_final['descripcion'].fillna('') + " " + df_final.get('features_text', pd.Series([""] * len(df_final))).fillna('')

def extraer_m2_pileta(texto):
    if pd.isna(texto): return 0.0
    texto = str(texto).lower()
    
    # Patrón explícito: "pileta de 28 m2"
    match_m2 = re.search(r'(?:pileta|piscina|espejo de agua).*?(\d+(?:[\.,]\d+)?)\s*(?:m2|mts|metros)', texto)
    if match_m2:
        return float(match_m2.group(1).replace(',', '.'))
    
    # Patrón medidas: "pileta de 7x3"
    match_medidas = re.search(r'(?:pileta|piscina).*?(\d+(?:[\.,]\d+)?)\s*[xX]\s*(\d+(?:[\.,]\d+)?)', texto)
    if match_medidas:
        largo = float(match_medidas.group(1).replace(',', '.'))
        ancho = float(match_medidas.group(2).replace(',', '.'))
        return largo * ancho
        
    return 0.0

def extraer_estado_texto(texto):
    if pd.isna(texto): return 'standard'
    texto = str(texto).lower()
    
    # Jerarquía: De lo más específico a lo general
    if any(k in texto for k in ['a estrenar', 'estrenar', 'recién terminada', 'posesión inmediata']):
        return 'nuevo'
    if any(k in texto for k in ['reciclada', 'remodelada', 'renovada', 'refaccionada a nuevo']):
        return 'remodelado'
    if any(k in texto for k in ['a refaccionar', 'refaccionar', 'demoler', 'para reciclar', 'humedad', 'deteriorad']):
        return 'refaccionar'
    if any(k in texto for k in ['muy buen estado', 'excelente estado', 'impecable']):
        return 'excelente'
        
    return 'standard'

# Solo busca m2 si ya sabemos que tiene pileta
df_final['pileta_m2_real'] = df_final.apply(
    lambda row: extraer_m2_pileta(row['texto_analisis']) if row.get('tiene_pileta', 0) == 1 else 0.0, axis=1
)

# JARDÍN (Binario)
keywords_jardin = ['jardín', 'jardin', 'parquizado', 'espacio verde', 'fondo libre', 'patio verde']
df_final['tiene_jardin'] = df_final['texto_analisis'].apply(
    lambda x: 1 if any(k in str(x).lower() for k in keywords_jardin) else 0
)

# ESTADO (Ordinal - Numérico)
df_final['estado_desc'] = df_final['texto_analisis'].apply(extraer_estado_texto)

In [91]:
def detectar_vista_mar(texto):
    if pd.isna(texto): return 0
    t = str(texto).lower()
    
    # Palabras estrictas de VISUAL o UBICACIÓN PRIMERA LÍNEA
    keywords_vista = [
        'frente al mar', 'salida al mar', 'salida a la playa', 
        'primera linea', 'primera línea', '1ra linea', 
        'sobre el mar', 'sobre la playa', 'pie en la arena',
        'vista al mar', 'vistas al mar', 'vista a la playa', 
        'vista al oceano', 'se ve el mar', 'vista plena al mar',
        'vista lateral al mar', 'vista mar'
    ]
    
    if any(k in t for k in keywords_vista):
        return 1
    return 0

# FUNCIÓN: ES CERCANO 
def detectar_cercania_mar(texto):
    if pd.isna(texto): return 0
    t = str(texto).lower()
    
    # Palabras de PROXIMIDAD FÍSICA (sin necesariamente tener vista)
    keywords_cerca = [
        'a metros del mar', 'a pocos metros del mar',
        'pasos del mar', 'a pasos de la playa', 
        'cerca del mar', 'cerca de la playa', 'muy cerca del mar',
        'a 1 cuadra del mar', 'a una cuadra del mar',
        'a 2 cuadras del mar', 'a dos cuadras del mar',
        'a 3 cuadras del mar', 'a tres cuadras del mar',
        'bajada a la playa', 'a pocas cuadras del mar',
        'zona mar', 'lado mar'
    ]
    
    if any(k in t for k in keywords_cerca):
        return 1
    return 0


cols_texto = [c for c in ['ubicacion', 'direccion', 'titulo', 'descripcion', 'texto_analisis'] if c in df_final.columns]
df_final['texto_full_mar'] = df_final[cols_texto].fillna('').agg(' '.join, axis=1)

df_final['tiene_vista_mar'] = df_final['texto_full_mar'].apply(detectar_vista_mar)
df_final['es_cercano_mar'] = df_final['texto_full_mar'].apply(detectar_cercania_mar)

print("\nDistribución de VISTA al Mar:")
print(df_final['tiene_vista_mar'].value_counts())

print("\nDistribución de CERCANÍA al Mar:")
print(df_final['es_cercano_mar'].value_counts())


Distribución de VISTA al Mar:
tiene_vista_mar
0    1237
1      51
Name: count, dtype: int64

Distribución de CERCANÍA al Mar:
es_cercano_mar
0    1193
1      95
Name: count, dtype: int64


In [92]:
def extractor_confort_termico(texto):
    if pd.isna(texto): return pd.Series({'gas_natural': 0, 'losa_radiante': 0})
    t = str(texto).lower()
    features = {}
    
    # GAS NATURAL 
    features['gas_natural'] = 1 if 'gas natural' in t or 'todos los servicios' in t else 0
    
    # CALEFACCIÓN CENTRAL
    features['losa_radiante'] = 1 if 'losa radiante' in t or 'calefacción central' in t or 'radiadores' in t or 'calefaccion central' in t else 0
    
    return pd.Series(features)

col_origen = 'texto_analisis' if 'texto_analisis' in df_final.columns else 'descripcion'

df_features = df_final[col_origen].apply(extractor_confort_termico)

cols_nuevas = ['gas_natural', 'losa_radiante']
df_final.drop(columns=[c for c in cols_nuevas if c in df_final.columns], inplace=True)

df_final = pd.concat([df_final, df_features], axis=1)

print(f"Con Gas Natural:   {df_final['gas_natural'].sum()} propiedades ({df_final['gas_natural'].mean():.1%})")
print(f"Con Losa Radiante: {df_final['losa_radiante'].sum()} propiedades ({df_final['losa_radiante'].mean():.1%})")

Con Gas Natural:   545 propiedades (42.3%)
Con Losa Radiante: 356 propiedades (27.6%)


In [93]:
df_final['porcentaje_cubierto'] = (df_final['m2_construidos_final'] / df_final['m2'].replace(0, 1)) * 100
df_final['porcentaje_cubierto'] = df_final['porcentaje_cubierto'].clip(upper=300)

def detectar_ph_loft(row):
    texto = (str(row.get('descripcion', '')) + " " + str(row.get('titulo', ''))).lower()
    if any(k in texto for k in [' ph ', ' ph.', ' ph,', 'propiedad horizontal', 'loft']):
        return 1
    return 0

df_final['es_ph_loft'] = df_final.apply(detectar_ph_loft, axis=1)
print(f" PHs/Lofts detectados: {df_final['es_ph_loft'].sum()}")

 PHs/Lofts detectados: 25


In [94]:
def detectar_antiguedad(row):
    texto = (str(row.get('descripcion', '')) + " " + str(row.get('caracteristica', ''))).lower()
    
    if any(k in texto for k in ['estrenar', 'moderna', 'minimalista', 'hormigón visto', 'hormigon visto', 'contemporánea']):
        return 'Moderna/Nueva'
        
    if any(k in texto for k in ['reciclad', 'refaccionad', 'renovad', 'hecha a nuevo', 'a nuevo']):
        return 'Renovada'
        
    if any(k in texto for k in ['clásica', 'clasica', 'tradicional', 'de época', 'de epoca', 'a reciclar', 'original', 'chalet', 'antigua', 'años 80', 'años 90']):
        return 'Clásica/Antigua'

    return 'Estándar/No Especifica'

df_final['estilo_antiguedad'] = df_final.apply(detectar_antiguedad, axis=1)

print("\nDistribución por Época/Estilo:")
print(df_final['estilo_antiguedad'].value_counts())

print("\nPrecio promedio por época:")
print(df_final.groupby('estilo_antiguedad')['precio'].mean().sort_values(ascending=False).apply(lambda x: f"${x:,.0f}"))


Distribución por Época/Estilo:
estilo_antiguedad
Estándar/No Especifica    912
Moderna/Nueva             201
Clásica/Antigua           133
Renovada                   42
Name: count, dtype: int64

Precio promedio por época:
estilo_antiguedad
Moderna/Nueva             $436,866
Estándar/No Especifica    $383,470
Clásica/Antigua           $303,402
Renovada                  $268,071
Name: precio, dtype: object


In [95]:
def detectar_tipo_especifico(row):
    texto = str(row.get('titulo', '')) + " " + str(row.get('descripcion', '')) + " " + str(row.get('caracteristica', ''))
    t = texto.lower()
    
    if re.search(r'\b(duplex|dúplex)\b', t): return 'Duplex'
    if re.search(r'\b(triplex|tríplex)\b', t): return 'Triplex'
    if re.search(r'\b(ph|p\.h\.|propiedad horizontal|tipo casa)\b', t): return 'PH'
    
    if re.search(r'\b(cabaña|cabana|alpina|log home)\b', t): return 'Cabaña'
    if re.search(r'\b(chalet|chalé|techo de teja)\b', t): return 'Chalet'
    if re.search(r'\b(mansion|residencia|villa)\b', t): return 'Mansión'
    
    if re.search(r'\b(departamento|depto|piso|semipiso|monoambiente)\b', t): return 'Departamento'

    if re.search(r'\b(casa|vivienda unifamiliar)\b', t): return 'Casa'

    if re.search(r'\b(propiedad|inmueble)\b', t):
        if re.search(r'\b(lote|parque|jardin|fondo)\b', t):
            return 'Casa'

    # Si menciona "Metros Cubiertos" Y "Metros Lote" -> Es CASA (Los deptos no tienen lote propio)
    if 'm2 cub' in t and 'lote' in t:
        return 'Casa'
        
    # Si menciona "Casa" implícitamente por características de exterior
    if re.search(r'\b(parque|jardín|fondo libre|quincho independiente)\b', t):
        return 'Casa'

    return None

df_final['tipo_detectado'] = df_final.apply(detectar_tipo_especifico, axis=1)

print("\nDistribución de Tipos Detectados:")
conteo = df_final['tipo_detectado'].value_counts(dropna=False)
print(conteo)

print(f"\nCantidad de 'No Detectados' (NaN): {df_final['tipo_detectado'].isna().sum()}")


Distribución de Tipos Detectados:
tipo_detectado
Casa            903
Departamento    126
Chalet          106
Duplex           45
None             36
Mansión          34
PH               18
Triplex          12
Cabaña            8
Name: count, dtype: int64

Cantidad de 'No Detectados' (NaN): 36


In [96]:
def imputar_tipo_preciso(row):
    # Valores actuales
    tipo_actual = str(row['tipo_detectado'])
    m2_total = float(row.get('m2', 0)) if not pd.isna(row.get('m2')) else 0
    m2_cub = float(row.get('m2_construidos_final', 0)) if not pd.isna(row.get('m2_construidos_final')) else m2_total
    
    texto = (str(row.get('descripcion', '')) + " " + str(row.get('titulo', ''))).lower()
    
    # DEPTOS GIGANTES (Prioridad Máxima)
    if tipo_actual == 'Departamento' and m2_total > 250:
        return 'Casa' 
        
    # 2. HOTFIX 2: FALSAS CASAS (PH y Lofts escondidos)
    if any(k in texto for k in [' ph ', ' ph.', ' ph,', 'propiedad horizontal']):
        return 'PH'
        
    if 'loft' in texto:
        return 'Loft'

    # RESPETAR TIPO ORIGINAL (Si superó los Hotfixes)
    if tipo_actual != 'nan' and tipo_actual != 'None' and tipo_actual != '':
        return tipo_actual

    # BÚSQUEDA POR PALABRAS CLAVE (Para los que no tienen tipo)
    if 'duplex' in texto or 'dúplex' in texto: return 'Duplex'
    if 'chalet' in texto: return 'Chalet'
    if 'triplex' in texto: return 'Triplex'
    if 'edificio' in texto or 'semipiso' in texto or 'piso exclusivo' in texto: return 'Departamento'

    # LÓGICA DE BARRIOS EXCLUSIVOS
    barrio = str(row.get('barrio_detectado', '')).lower()
    
    zonas_casas = ['herradura', 'costa esmeralda', 'villarobles', 'cariló', 'carilo', 'golf', 'pioneros', 'norte']
    if any(z in barrio for z in zonas_casas):
        if m2_total < 55: return 'Departamento'
        return 'Casa'

    # Diferencia Total vs Cubierto
    espacio_aire = m2_total - m2_cub
    if espacio_aire > 60:
        return 'Casa'

    # REGLAS POR TAMAÑO (Último recurso)
    if m2_total > 150: 
        return 'Casa'
    
    if m2_total < 60: 
        return 'Departamento'
        
    if 'centro' in barrio or 'duplex' in barrio:
        if m2_total < 100: return 'Departamento'

    # Default final
    return 'Casa'

df_final['tipo_detectado'] = df_final.apply(imputar_tipo_preciso, axis=1)

print("\nDistribución Final:")
print(df_final['tipo_detectado'].value_counts())


Distribución Final:
tipo_detectado
Casa            1037
Chalet           106
Duplex            42
Mansión           33
Departamento      24
PH                20
Triplex           11
Cabaña             8
Loft               7
Name: count, dtype: int64


In [97]:
def detectar_barrio_v3(texto):
    if pd.isna(texto): return "Pinamar Centro"
    t = str(texto).lower()
    
    if any(k in t for k in ['costa esmeralda', 'villarobles', 'villa robles', 'northbeach']):
        return 'Costa Esmeralda' 
    if 'herradura' in t:
        return 'La Herradura'
    if 'carilo' in t or 'cariló' in t:
        return 'Cariló'

    keywords_norte = ['pinamar norte', 'zona norte', 'la frontera', 'barrio marítimo', 'barrio maritimo']
    if any(k in t for k in keywords_norte):
        return 'Pinamar Norte'
    
    if any(k in t for k in ['barrio golf', 'zona golf', 'isla del golf', 'cancha de golf']):
        return 'Golf'
    
    if any(k in t for k in ['los alamos', 'los álamos', 'pioneros', 'nayades', 'náyades', 'barrio parque']):
        return 'Alamos/Pioneros'

    if 'valeria' in t:
        return 'Valeria del Mar'
    if 'ostende' in t:
        return 'Ostende'
    if any(k in t for k in ['zona duplex', 'zona centro', 'pinamar centro']):
        return 'Pinamar Centro'

    return 'Pinamar General' 

# REFINADOR POR CALLES
def refinar_por_calle(row):
    if row['barrio_detectado'] != 'Pinamar General':
        return row['barrio_detectado']
    
    t = (str(row.get('direccion', '')) + " " + str(row.get('descripcion', ''))).lower()
    
    if any(calle in t for calle in ['olimpo', 'aquiles', 'troya', 'ilíada', 'iliada', 'odisea', 'penélope', 'penelope', 'ulises', 'libertador norte']):
        return 'Pinamar Norte'
    if any(calle in t for calle in ['acacias', 'eucaliptus', 'eucaliptos', 'aromos', 'cipreses', 'álamos', 'alamos']):
        return 'Alamos/Pioneros'
    if any(calle in t for calle in ['caracol', 'medusas', 'tridente', 'sirena', 'caracoles', 'del mar', 'del odiseo']):
        return 'Pinamar Centro' 
    if any(calle in t for calle in ['bunge', 'simbad', 'artes', 'musas', 'libertador sur', 'jasón', 'jason']):
        return 'Pinamar Centro'
    if any(calle in t for calle in ['goleta', 'fragata', 'corbeta', 'carabela']):
        return 'Golf'

    return 'Pinamar General'


cols_ubicacion = ['ubicacion', 'direccion', 'lugar', 'titulo', 'descripcion', 'caracteristica']
cols_reales = [c for c in cols_ubicacion if c in df_final.columns]
df_final['texto_full_ubicacion'] = df_final[cols_reales].fillna('').agg(' '.join, axis=1)

df_final['barrio_detectado'] = df_final['texto_full_ubicacion'].apply(detectar_barrio_v3)
df_final['barrio_detectado'] = df_final.apply(refinar_por_calle, axis=1)

print("\nDistribución Final de Barrios:")
print(df_final['barrio_detectado'].value_counts())


Distribución Final de Barrios:
barrio_detectado
Pinamar General    304
Pinamar Norte      254
Pinamar Centro     218
Alamos/Pioneros    160
La Herradura       118
Golf                91
Costa Esmeralda     81
Ostende             24
Cariló              22
Valeria del Mar     16
Name: count, dtype: int64


In [98]:
col_total = 'm2'
col_cubierta = 'm2_construidos_final' if 'm2_construidos_final' in df_final.columns else 'm2_cubiertos'
  
df_final['m2_no_construidos'] = df_final[col_total] - df_final[col_cubierta]
    
# CORREGIR ERRORES (Negativos)
errores_negativos = (df_final['m2_no_construidos'] < 0).sum()
df_final['m2_no_construidos'] = df_final['m2_no_construidos'].apply(lambda x: max(0, x))
    
print(f"Se corrigieron {errores_negativos} casos donde la superficie construida superaba al terreno (se forzaron a 0).")    
print("\nDistribución de Espacio Libre:")
print(df_final['m2_no_construidos'].describe().round(1))


Se corrigieron 1 casos donde la superficie construida superaba al terreno (se forzaron a 0).

Distribución de Espacio Libre:
count        1288.0
mean        13797.6
std        344432.9
min             0.0
25%           199.2
50%           589.5
75%           855.0
max      10779244.0
Name: m2_no_construidos, dtype: float64


In [99]:
def detector_lujo_hardware(row):
    texto = str(row.get('texto_analisis', '')) + " " + str(row.get('features_text', ''))
    t = texto.lower()
    
    score = 0
    
    # ESTRUCTURA Y TECNOLOGÍA / definen una casa de alta gama moderna
    triggers_fuertes = [
        'losa radiante', 'piso radiante', 'calefacción central', 'calefaccion central',
        'aire central', 'vrv',            # Climatización central
        'dvh', 'doble vidrio',            # Aislación
        'aberturas pvc', 'aberturas a30', 'aberturas a40', # Carpintería Premium
        'ascensor', 'domótica', 'casa inteligente',
        'hormigón visto', 'diseño de autor'
    ]
    
    #AMENITIES Y TERMINACIONES / Son lujos, pero por sí solos no garantizan que toda la casa sea premium
    triggers_medios = [
        'jacuzzi', 'hidromasaje', 'sauna', 'finlandés', 'ducha escocesa', 'baño turco',
        'pileta climatizada', 'piscina climatizada', 'in-out',
        'cava', 'cine', 'microcine', 
        'riego por aspersión',
        'silestone', 'neolith', 'mármol', 'marmol', # Mesadas caras
        'porcelanato', 'madera maciza', 'lapacho', 'incienso', 
        'suite con vestidor'
    ]


    encontrados_fuertes = [k for k in triggers_fuertes if k in t]
    if len(encontrados_fuertes) > 0:
        score += 2 * len(set(encontrados_fuertes))

    encontrados_medios = [k for k in triggers_medios if k in t]
    if len(encontrados_medios) > 0:
        score += 1 * len(set(encontrados_medios)) 

    # UMBRAL DE LUJO - Necesita 3 puntos.
    return 1 if score >= 3 else 0

df_final['es_lujosa_hardware'] = df_final.apply(detector_lujo_hardware, axis=1)
total_lujo = df_final['es_lujosa_hardware'].sum()
porcentaje = (total_lujo / len(df_final)) * 100

print(f"Propiedades detectadas como Lujo: {total_lujo} ({porcentaje:.1f}%)")
print(f"Propiedades Estándar:             {len(df_final) - total_lujo}")

print("\nEJEMPLOS DETECTADOS:")
ejemplos = df_final[df_final['es_lujosa_hardware'] == 1].sample(min(3, len(df_final[df_final['es_lujosa_hardware'] == 1])))

for idx, row in ejemplos.iterrows():
    print(f"ID {idx} | Precio: {row['precio']} USD")
    texto = (str(row.get('texto_analisis', '')) + " " + str(row.get('features_text', ''))).lower()
    
    triggers_ver = ['losa radiante', 'dvh', 'calefacción central', 'jacuzzi', 'porcelanato', 'silestone', 'pvc']
    encontrados = [t for t in triggers_ver if t in texto]
    
    print(f"DETECTADO POR: {list(set(encontrados))}")
    print(f"Fragmento: {row['descripcion'][:150]}...")

Propiedades detectadas como Lujo: 275 (21.4%)
Propiedades Estándar:             1013

EJEMPLOS DETECTADOS:
ID 123 | Precio: 420000.0 USD
DETECTADO POR: ['pvc', 'losa radiante', 'dvh']
Fragmento: Casa en venta en Pinamar, barrio Los Pioneros. Una casa diseñada y construida a conciencia con materiales de primera calidad. Su distribución se desar...
ID 549 | Precio: 335000.0 USD
DETECTADO POR: ['porcelanato']
Fragmento: Temporada 2026: valores por quincena Y mes en detalle del aviso1ra quincena de enero (usd) 8. 5002da quincena de enero (usd) 7. 5001ra quincena de feb...
ID 456 | Precio: 330000.0 USD
DETECTADO POR: ['pvc', 'dvh']
Fragmento: Oilher Propiedades presenta a la venta esta increible casa en el barrio B5! La casa cuenta con 6 años de antiguedad, emplaada sobre un lote de 450m2 l...


In [100]:
df_final['ratio_bano_dormitorio'] = df_final['baños'] / df_final['dormitorios'].replace(0, 1)

# FACTOR DE PARQUE (Relación Terreno/Construido)
df_final['factor_parque'] = df_final['m2'] / df_final['m2_construidos_final'].replace(0, 1)

# 3. SEGURIDAD (Keywords)
def detectar_seguridad(texto):
    if pd.isna(texto): return 0
    t = str(texto).lower()
    keywords = ['seguridad', 'vigilancia', 'barrio cerrado', 'monitoreo', 'cerco electrico', 'guardia']
    if any(k in t for k in keywords):
        return 1
    return 0

df_final['tiene_seguridad'] = df_final['texto_completo'].apply(detectar_seguridad)

# MODERNIDAD / A ESTRENAR
def detectar_modernidad(texto):
    if pd.isna(texto): return 0
    t = str(texto).lower()
    keywords = ['a estrenar', 'estrenar', 'minimalista', 'moderna', 'hormigon visto', 'hormigón visto', 'diseño de autor', 'concepto abierto']
    if any(k in t for k in keywords):
        return 1
    return 0

df_final['es_moderna_estrenar'] = df_final['texto_completo'].apply(detectar_modernidad)

print(df_final[['ratio_bano_dormitorio', 'factor_parque', 'tiene_seguridad', 'es_moderna_estrenar']].head())

   ratio_bano_dormitorio factor_parque  tiene_seguridad  es_moderna_estrenar
0                  0.875       4.62963                0                    1
1                  0.750      6.718894                1                    0
2                  0.750     12.914286                0                    0
3                  0.800      4.605948                1                    0
4                  1.000           4.0                0                    0


In [101]:
def extraer_variables_premium(row):
    texto = (str(row.get('descripcion', '')) + " " + str(row.get('caracteristica', '')) + " " + str(row.get('titulo', ''))).lower()

    menciona_frase_exacta = any(f in texto for f in ['pileta climatizada', 'piscina climatizada'])
    climatizada = 1 if  menciona_frase_exacta else 0
    
    dependencia = 1 if any(k in texto for k in ['cuarto de servicio', 'habitación de servicio', 'habitacion de servicio']) else 0
    
    dvh = 1 if any(k in texto for k in ['dvh', 'doble vidrio', 'aberturas de pvc', 'termopanel']) else 0
    
    cochera_cubierta = 1 if any(k in texto for k in ['cochera cubierta', 'garage cerrado',  'garaje cerrado']) else 0
    
    return pd.Series([climatizada, dependencia, dvh, cochera_cubierta])

nuevas_cols = ['pileta_climatizada', 'dependencia_servicio', 'doble_vidrio_dvh', 'cochera_cerrada']
df_final[nuevas_cols] = df_final.apply(extraer_variables_premium, axis=1)

print("\nImpacto de la nueva extracción:")
for col in nuevas_cols:
    cant = df_final[col].sum()
    print(f"{col:<25}: {cant:>3} casas detectadas.")


Impacto de la nueva extracción:
pileta_climatizada       : 122 casas detectadas.
dependencia_servicio     :  62 casas detectadas.
doble_vidrio_dvh         : 233 casas detectadas.
cochera_cerrada          : 172 casas detectadas.


In [102]:
df_final['nivel_pileta'] = 0
df_final.loc[df_final['tiene_pileta'] == 1, 'nivel_pileta'] = 1
df_final.loc[df_final['pileta_climatizada'] == 1, 'nivel_pileta'] = 2

#Borra las dos columnas viejas
columnas_a_borrar = ['tiene_pileta', 'pileta_climatizada']
df_final = df_final.drop(columns=columnas_a_borrar, errors='ignore')

print("\nDistribución de Nivel de Pileta:")
conteos = df_final['nivel_pileta'].value_counts().sort_index()
print(f"Nivel 0 (Sin Pileta)         : {conteos.get(0, 0)} propiedades")
print(f"Nivel 1 (Pileta Estándar)    : {conteos.get(1, 0)} propiedades")
print(f"Nivel 2 (Pileta Climatizada) : {conteos.get(2, 0)} propiedades")


Distribución de Nivel de Pileta:
Nivel 0 (Sin Pileta)         : 786 propiedades
Nivel 1 (Pileta Estándar)    : 380 propiedades
Nivel 2 (Pileta Climatizada) : 122 propiedades


In [103]:
df_final['porcentaje_cubierto'] = (df_final['m2_construidos_final'] / df_final['m2'].replace(0, 1)) * 100

# Filtro de seguridad: Si por error da más del 100% 
df_final['porcentaje_cubierto'] = df_final['porcentaje_cubierto'].clip(upper=300)

print("\nMuestra de las nuevas proporciones:")
print(df_final[['m2', 'm2_construidos_final', 'porcentaje_cubierto']].head())

corr_pct = df_final['porcentaje_cubierto'].corr(df_final['precio'])
print(f"\nCorrelación con el precio: {corr_pct:+.3f}")


Muestra de las nuevas proporciones:
       m2  m2_construidos_final porcentaje_cubierto
0  1125.0                 243.0                21.6
1  1458.0                 217.0           14.883402
2  2260.0                 175.0            7.743363
3  1239.0                 269.0           21.711057
4   300.0                  75.0                25.0

Correlación con el precio: -0.172


In [104]:
def detectar_renovada(row):
    texto = (str(row.get('descripcion', '')) + " " + str(row.get('titulo', ''))).lower()
    
    keywords_positivas = [
        'reciclad', 'renovad', 'refaccionad', 'a nuevo', 
        'hecha a nuevo', 'hecho a nuevo', 'cañerias nuevas', 
        'instalaciones nuevas', 'puesto a nuevo', 'puesta a nuevo',
        'remodelad', 'restaurad', 'impecable estado'
    ]
    
    # Frases trampa (Falsos positivos)
    frases_negativas = ['para reciclar', 'a reciclar', 'a refaccionar', 'para refaccionar', 'a remodelar']
    if any(neg in texto for neg in frases_negativas):
        return 0
        
    if any(pos in texto for pos in keywords_positivas):
        return 1
        
    return 0

def detectar_complejo(row):
    texto = (str(row.get('descripcion', '')) + " " + str(row.get('titulo', ''))).lower()
    
    keywords_complejo = [
        'complejo', 'condominio', 'housing', 'unidades funcionales', 
        'uf ', 'compartido', 'espacios comunes', 'ph al fondo', 'complejo de'
    ]
    
    if any(k in texto for k in keywords_complejo):
        return 1
    return 0
df_final['es_renovada'] = df_final.apply(detectar_renovada, axis=1)
df_final['en_complejo'] = df_final.apply(detectar_complejo, axis=1)


cant_renov = df_final['es_renovada'].sum()
precio_renov = df_final[df_final['es_renovada']==1]['precio'].median()
precio_normal = df_final[df_final['es_renovada']==0]['precio'].median()

print(f"Renovadas/Recicladas detectadas: {cant_renov}")
if cant_renov > 0:
    print(f"Mediana Renovada: ${precio_renov:,.0f} vs Normal: ${precio_normal:,.0f}")
    diff = ((precio_renov - precio_normal) / precio_normal) * 100
    print(f"Diferencia de precio: {diff:+.1f}%")

cant_complejo = df_final['en_complejo'].sum()
print(f"\nPropiedades en Complejo/Housing: {cant_complejo}")

Renovadas/Recicladas detectadas: 52
Mediana Renovada: $237,500 vs Normal: $320,000
Diferencia de precio: -25.8%

Propiedades en Complejo/Housing: 107


In [105]:
# LA DIFERENCIA (Espacios Sociales Extra)
# Nos dice cuántos espacios "no-dormitorio" tiene la casa.
# Valores altos = Mansión con múltiples salas.
df_final['ambientes_extra'] = df_final['ambientes'] - df_final['dormitorios']

# EL RATIO (Proporción)
# Qué porcentaje de la casa se destina a dormir vs a vivir.
df_final['ratio_dormitorio_ambiente'] = df_final['dormitorios'] / df_final['ambientes'].replace(0, 1)

#Más dormitorios que ambientes (Imposible físico)
anomalias_graves = df_final[df_final['ambientes_extra'] < 0]
print(f"{len(anomalias_graves)} casos imposibles (Dormitorios > Ambientes).")

#Dormitorios = Ambientes (Muy raro, no hay living)
anomalias_leves = df_final[df_final['ambientes_extra'] == 0]
print(f"{len(anomalias_leves)} casos sospechosos")

4 casos imposibles (Dormitorios > Ambientes).
15 casos sospechosos


In [106]:
# ARREGLAR CASOS
mask_errores = df_final['ambientes_extra'] <= 0
# Forzar que los ambientes sean los dormitorios + 1 sala de estar
df_final.loc[mask_errores, 'ambientes'] = df_final.loc[mask_errores, 'dormitorios'] + 1

df_final['ambientes_extra'] = df_final['ambientes'] - df_final['dormitorios']
df_final['ratio_dormitorio_ambiente'] = df_final['dormitorios'] / df_final['ambientes'].replace(0, 1)

In [107]:
def extraer_cantidad_plantas(row):
    texto = (str(row.get('descripcion', '')) + " " + 
             str(row.get('caracteristica', '')) + " " + 
             str(row.get('titulo', ''))).lower()
    
    if any(k in texto for k in ['3 plantas', 'tres plantas', '3 niveles', 'tres niveles', 'triplex', 'tríplex']):
        return 3
        
    if any(k in texto for k in ['2 plantas', 'dos plantas', '2 niveles', 'dos niveles', 'duplex', 'dúplex', 'planta alta', 'pa y pb']):
        return 2
        
    if any(k in texto for k in ['1 planta', 'una planta', 'una sola planta', 'desarrollada en pb', 'toda en planta baja', 'todo en planta baja']):
        return 1
        
    return 1

df_final['cantidad_plantas'] = df_final.apply(extraer_cantidad_plantas, axis=1)

print("\nDistribución de Pisos/Plantas:")
print(df_final['cantidad_plantas'].value_counts().sort_index())
print("\nPrecio promedio por Cantidad de Plantas:")
print(df_final.groupby('cantidad_plantas')['precio'].mean().sort_values(ascending=False).apply(lambda x: f"${x:,.0f}"))


Distribución de Pisos/Plantas:
cantidad_plantas
1    644
2    579
3     65
Name: count, dtype: int64

Precio promedio por Cantidad de Plantas:
cantidad_plantas
3    $484,923
2    $390,866
1    $359,184
Name: precio, dtype: object


In [108]:
df_final.to_csv('../data/preproccessed-props-pinamar.csv', index=False)
#df_final.to_csv('../data/preproccessed-pinamar-test.csv', index=False)

print(f"Total de Propiedades: {len(df_final)}")
print(f"Total de Columnas: {len(df_final.columns)}")
print("\nTipos de datos:")
print(df_final.dtypes)

Total de Propiedades: 1288
Total de Columnas: 45

Tipos de datos:
precio                       float64
m2                            object
ambientes                    float64
dormitorios                    int64
baños                          int64
cocheras                     float64
direccion                     object
descripcion                   object
caracteristica                object
features_text                 object
texto_completo                object
es_a_refaccionar               int64
tiene_quincho                float64
m2_cubiertos_extract         float64
m2_construidos_final         float64
texto_analisis                object
pileta_m2_real               float64
tiene_jardin                   int64
estado_desc                   object
texto_full_mar                object
tiene_vista_mar                int64
es_cercano_mar                 int64
gas_natural                    int64
losa_radiante                  int64
porcentaje_cubierto           object
es_ph_lof